In [ ]:
import pandas as pd
import json
from datetime import datetime
# read Excel file
df = pd.read_excel('test1.xlsx')
# get columns 
employee = df['emp_id']
name = df['emp_name']
code = df['scr_selcode']
classroom = df['cls_name']
subject = df['sub_name']
location = df['rom_name']
t_date = df['TEST_DATE']
t_time = df['TEST_TIME']

# rearrange columns data
data = pd.concat([employee, name, code, classroom, subject, location, t_date, t_time], axis=1)
data.columns = ['Employee_ID', 'Employee_Name', 'Course_Code', 'Class', 'Subject', 'Location', 'Test_Date', 'Test_Time']

# Check data type
print(type(data))

In [ ]:
# Make the Course_Code columns into a string type
data['Course_Code']=data['Course_Code'].astype(str)

# Get data length
length = len(data)

# Convert the course code to four numbers
for i in range(len(data['Course_Code'])):
    coursecode = data['Course_Code'][i]
    if len(coursecode)==3:
        coursecode = '0' + coursecode
        data['Course_Code'][i] = coursecode
for i in range(len(data['Course_Code'])):
    coursecode = data['Course_Code'][i]
    if len(coursecode)==2:
        coursecode = '00' + coursecode
        data['Course_Code'][i] = coursecode
for i in range(len(data['Course_Code'])):
    coursecode = data['Course_Code'][i]
    if len(coursecode)==1:
        coursecode = '000' + coursecode
        data['Course_Code'][i] = coursecode
# Checking Course_Code 
print(data['Course_Code'])

In [ ]:
# Convert Test_Time to 'Hour and Minutes' format
for j in range(len(data['Test_Time'])):
    ttime=data['Test_Time'][j]
    if type(ttime)!=float:
        timeObject=ttime.strftime("%H:%M")
    else:
        timeObject=" "
    data['Test_Time'].astype(str)
    data['Test_Time'][j]=timeObject
# Checking Test_Time
print(data['Test_Time'])

In [ ]:
# Convert Test_Date to 'Month and Day' format
for k in range(len(data['Test_Date'])):
    ddate=data['Test_Date'][k]
    if type(ddate)!=type(pd.NaT):
        dateObj = ddate.strftime("%m/%d")
    else:
        dateObj=" "
    data['Test_Date'].astype(str)
    data['Test_Date'][k]=dateObj
# Checking Test_Date
print(data['Test_Date'])

In [ ]:
# Checking Employee_Name columns if is NaN replace to space
for l in range(len(data['Employee_Name'])):
    teachName=data['Employee_Name'][l]
    if type(teachName)==float:
        nameObj=" "
    elif type(teachName)==str:
        nameObj=teachName
    data['Employee_Name'][l]=nameObj
# Checking Test_Date
print(data['Employee_Name'])

In [ ]:
#Before Upload to Firebase Realtime Database 
#Print out all data for checking
print(data)

In [ ]:
# import function
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Firebase Admin SDK
# Admin SDK configuration code
cred = credentials.Certificate("qa-insurance-firebase-adminsdk-rlu56-b9b0fce82a.json")
default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://qa-insurance.firebaseio.com/'
})

# Before Upload to Firebase RealTime Database need to delete all data.
# Because there using push function.
# Delete all data.
path = "/"
doc_ref = db.reference(path);
doc_ref.delete()

In [ ]:
# import function
from firebase_admin import db

# Definition Class for Upload data to Firebase by employee name.
def setDatabase(cls, course, td, ids, location, names, subject, tt):
    ref = db.reference('/')
    users_ref = ref.child(names)
    users_ref.push({
        "CLASS": cls,
        "COURSE": course,
        "DATE": td,
        "ID": ids,
        "LOCATION": location,
        "NAME": names,
        "SUBJECT": subject,
        "TIME": tt
    })

In [ ]:
# import function
from firebase_admin import db

# Definition Class for Upload data to Firebase by course code.
def setDatabaseByCode(cls, course, td, ids, location, names, subject, tt):
    ref = db.reference('/')
    users_ref = ref.child(ids)
    users_ref.push({
        "CLASS": cls,
        "COURSE": course,
        "DATE": td,
        "ID": ids,
        "LOCATION": location,
        "NAME": names,
        "SUBJECT": subject,
        "TIME": tt
    })

In [ ]:
# import function
from firebase_admin import db

# Definition Class for Upload data to Firebase by employee id.
def setDatabaseByIDS(cls, course, td, ids, location, names, subject, tt):
    ref = db.reference('/')
    users_ref = ref.child(course)
    users_ref.push({
        "CLASS": cls,
        "COURSE": course,
        "DATE": td,
        "ID": ids,
        "LOCATION": location,
        "NAME": names,
        "SUBJECT": subject,
        "TIME": tt
    })

In [ ]:
# Upload data by employee name
for i in range(length):
    setDatabase(str(data.Class[i]), str(data.Course_Code[i]), str(data.Test_Date[i]), str(data.Employee_ID[i]), str(data.Location[i]), str(data.Employee_Name[i]), str(data.Subject[i]), str(data.Test_Time[i]))

# Upload data by course code
for i in range(length):
    setDatabaseByCode(str(data.Class[i]), str(data.Course_Code[i]), str(data.Test_Date[i]), str(data.Employee_ID[i]), str(data.Location[i]), str(data.Employee_Name[i]), str(data.Subject[i]), str(data.Test_Time[i]))

# Upload data by employee ID
for i in range(length):
    setDatabaseByIDS(str(data.Class[i]), str(data.Course_Code[i]), str(data.Test_Date[i]), str(data.Employee_ID[i]), str(data.Location[i]), str(data.Employee_Name[i]), str(data.Subject[i]), str(data.Test_Time[i]))
    

In [ ]:
# import function.
from firebase_admin import db
import datetime

# Upload version date to firebase.
# Let user know when is the last Update
now = datetime.datetime.now()
dateStr = str(now.year) + '/' + str(now.month) + '/' + str(now.day)
# print(dateStr)
ref = db.reference('/version')
ref.set({
    "0": dateStr
})